In [29]:
import pandas as pd

# Loading the CSV file into a DataFrame
crsp_daily = pd.read_csv('./crsp_dret.csv')

In [30]:
# Sort by 'ticker' and 'date'
crsp_daily = crsp_daily.sort_values(by=['ticker', 'date'])

In [31]:
# Calculate summary statistics
summary_stats = crsp_daily[['ret', 'vol']].describe()

summary_stats = summary_stats.loc[['mean', '50%', 'std', 'count']]

summary_stats.rename(index={'50%': 'median'}, inplace=True)

print(summary_stats)

                 ret           vol
mean    5.402425e-04  1.364990e+06
median  0.000000e+00  2.616000e+05
std     5.333287e-02  5.526205e+06
count   1.794757e+06  1.795289e+06


In [32]:
from datetime import datetime

df = pd.read_csv('./robinhood_users_holding.csv')

# Convert timestamp
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Create new column
df['date'] = df['timestamp'].apply(lambda x: x.date().isoformat())

print(df)

                  timestamp  users_holding ticker        date
0       2018-05-03 14:45:59            587      A  2018-05-03
1       2018-05-03 15:46:00            586      A  2018-05-03
2       2018-05-03 16:46:00            587      A  2018-05-03
3       2018-05-03 17:45:59            587      A  2018-05-03
4       2018-05-05 14:45:59            588      A  2018-05-05
...                     ...            ...    ...         ...
7456372 2020-08-12 15:47:45           1475    OZK  2020-08-12
7456373 2020-08-12 16:47:28           1477    OZK  2020-08-12
7456374 2020-08-13 14:55:47           1480    OZK  2020-08-13
7456375 2020-08-13 15:54:20           1482    OZK  2020-08-13
7456376 2020-08-13 16:54:16           1482    OZK  2020-08-13

[7456377 rows x 4 columns]


In [33]:
# Extract hour and minute for filtering
df['hour'] = df['timestamp'].dt.hour
df['minute'] = df['timestamp'].dt.minute

# I extracted the hour and minute for filtering to avoid complications mixing the 'datetime64' and 'time' types

# Filter out entries from 4 PM or later
df_before_4pm = df[(df['hour'] < 16)]

# Group by 'ticker' and 'date'
user_daily = df_before_4pm.groupby(['ticker', 'date']).last().reset_index()

# Select the necessary columns and rename
user_daily = user_daily[['ticker', 'date', 'users_holding']]
user_daily.rename(columns={'users_holding': 'users_close'}, inplace=True)

# Calculate summary statistics for 'users_close'
summary_stats_users = user_daily['users_close'].describe()

print(user_daily)
print(summary_stats_users)

        ticker        date  users_close
0            A  2018-05-03          586
1            A  2018-05-05          588
2            A  2018-05-06          588
3            A  2018-05-07          582
4            A  2018-05-08          577
...        ...         ...          ...
1765425    OZK  2020-08-09         1491
1765426    OZK  2020-08-10         1487
1765427    OZK  2020-08-11         1480
1765428    OZK  2020-08-12         1475
1765429    OZK  2020-08-13         1482

[1765430 rows x 3 columns]
count    1.765430e+06
mean     3.098802e+03
std      2.118899e+04
min      0.000000e+00
25%      7.800000e+01
50%      2.820000e+02
75%      1.044000e+03
max      9.449220e+05
Name: users_close, dtype: float64


In [34]:
import gc

del df
collected = gc.collect()
print(f"{collected} objects freed from memory. Or you could install more RAM :)")

23 objects freed from memory. Or you could install more RAM :)


In [35]:
user_daily['date'] = pd.to_datetime(user_daily['date'])
user_daily.sort_values(by=['ticker', 'date'], inplace=True)

# Calculate 'userchg'
user_daily['userchg'] = user_daily.groupby('ticker')['users_close'].diff()

# Calculate 'abnormal_userchg'
rolling_mean = user_daily.groupby('ticker')['userchg'].transform(lambda x: x.rolling(window=5).mean())
user_daily['abnormal_userchg'] = user_daily['userchg'] - rolling_mean

# Calculate 'abnormal_userchg_lag'
user_daily['abnormal_userchg_lag'] = user_daily.groupby('ticker')['abnormal_userchg'].shift(1)

# Calculate summary statistics
summary_stats = user_daily[['userchg', 'abnormal_userchg', 'abnormal_userchg_lag']].describe()

print(summary_stats)

            userchg  abnormal_userchg  abnormal_userchg_lag
count  1.762864e+06      1.752608e+06          1.750045e+06
mean   1.016733e+01     -5.947388e-02         -5.409712e-02
std    2.652290e+02      1.965703e+02          1.960411e+02
min   -3.998100e+04     -4.259720e+04         -4.259720e+04
25%   -1.000000e+00     -1.400000e+00         -1.400000e+00
50%    0.000000e+00      0.000000e+00          0.000000e+00
75%    1.000000e+00      1.200000e+00          1.200000e+00
max    6.277900e+04      4.081540e+04          4.081540e+04


In [36]:
# Convert 'date' to datetime format in 'crsp_daily'
crsp_daily['date'] = pd.to_datetime(crsp_daily['date'])

# Merge the DataFrames on 'ticker' and 'date'
user_merged = pd.merge(user_daily, crsp_daily, on=['ticker', 'date'], how='inner')

# Sort the merged DataFrame by 'ticker' and 'date'
user_merged.sort_values(by=['ticker', 'date'], inplace=True)

# Calculate summary statistics for all variables except 'ticker' and 'date'
summary_stats_merged = user_merged.drop(columns=['ticker', 'date']).describe()

print(user_merged)
print(summary_stats_merged)

        ticker       date  users_close  userchg  abnormal_userchg  \
0            A 2018-05-03          586      NaN               NaN   
1            A 2018-05-07          582     -6.0               NaN   
2            A 2018-05-08          577     -5.0               NaN   
3            A 2018-05-09          582      5.0               5.8   
4            A 2018-05-10          584      2.0               2.8   
...        ...        ...          ...      ...               ...   
1197235    OZK 2020-08-07         1487     -2.0              -1.4   
1197236    OZK 2020-08-10         1487     -4.0              -3.6   
1197237    OZK 2020-08-11         1480     -7.0              -5.2   
1197238    OZK 2020-08-12         1475     -5.0              -2.6   
1197239    OZK 2020-08-13         1482      7.0               8.8   

         abnormal_userchg_lag        vol       ret  
0                         NaN  2365851.0  0.006524  
1                         NaN  1468735.0  0.005821  
2           

In [37]:
# Calculate the absolute return
user_merged['abs_return'] = user_merged['ret'].abs()

# Rank the absolute returns
user_merged['rank'] = user_merged.groupby('date')['abs_return'].rank(method='min', ascending=False)

# Create 'extreme_absolute_return'
user_merged['extreme_absolute_return'] = (user_merged['rank'] <= 20).astype(int)

# Create the lagged 'extreme_absolute_return'
user_merged['extreme_absolute_return_lag'] = user_merged.groupby('ticker')['extreme_absolute_return'].shift(1)

# Fill NaN values with 0 in the lagged column
user_merged['extreme_absolute_return_lag'] = user_merged['extreme_absolute_return_lag'].fillna(0)

print(user_merged[['date', 'ticker', 'abs_return', 'rank', 'extreme_absolute_return', 'extreme_absolute_return_lag']])

              date ticker  abs_return    rank  extreme_absolute_return  \
0       2018-05-03      A    0.006524  1327.0                        0   
1       2018-05-07      A    0.005821  1351.0                        0   
2       2018-05-08      A    0.000297  1862.0                        0   
3       2018-05-09      A    0.011281  1015.0                        0   
4       2018-05-10      A    0.010715   929.0                        0   
...            ...    ...         ...     ...                      ...   
1197235 2020-08-07    OZK    0.027530   943.0                        0   
1197236 2020-08-10    OZK    0.018519  1288.0                        0   
1197237 2020-08-11    OZK    0.016248  1224.0                        0   
1197238 2020-08-12    OZK    0.008755  1600.0                        0   
1197239 2020-08-13    OZK    0.004224  1960.0                        0   

         extreme_absolute_return_lag  
0                                0.0  
1                                

In [38]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Extract the year from the date
user_merged['year'] = pd.DatetimeIndex(user_merged['date']).year

# Define the regression model
formula = 'abnormal_userchg ~ extreme_absolute_return_lag + abnormal_userchg_lag + C(year)'

# Fit the regression model
model = smf.ols(formula, data=user_merged).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:       abnormal_userchg   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     7764.
Date:                Tue, 07 May 2024   Prob (F-statistic):               0.00
Time:                        05:01:10   Log-Likelihood:            -8.0448e+06
No. Observations:             1187032   AIC:                         1.609e+07
Df Residuals:                 1187027   BIC:                         1.609e+07
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept         

In [39]:
# Drop rows where 'ret' or 'abnormal_userchg_lag' are NaN
user_merged = user_merged.dropna(subset=['ret', 'abnormal_userchg_lag'])

print(user_merged.shape)

(1186896, 13)


In [40]:
user_merged = user_merged.copy()

# Assign deciles to 'abnormal_userchg_lag' for each date
# Create a new column 'decile'
user_merged['decile'] = user_merged.groupby('date')['abnormal_userchg_lag'].transform(
    lambda x: pd.qcut(x, 10, labels=False, duplicates='drop') + 1
)

# Calculate the average return
decile_returns = user_merged.groupby(['date', 'decile'])['ret'].mean().reset_index()

# Rename columns
decile_returns.rename(columns={'ret': 'avg_return'}, inplace=True)

print(decile_returns)

           date  decile  avg_return
0    2018-05-10       1    0.009156
1    2018-05-10       2    0.006445
2    2018-05-10       3    0.005149
3    2018-05-10       4    0.004445
4    2018-05-10       5    0.003009
...         ...     ...         ...
5340 2020-08-13       6   -0.003980
5341 2020-08-13       7    0.000780
5342 2020-08-13       8   -0.003462
5343 2020-08-13       9    0.010984
5344 2020-08-13      10   -0.004436

[5345 rows x 3 columns]


In [41]:
# Pivot the table to wide format
wide_format = decile_returns.pivot(index='date', columns='decile', values='avg_return')

# Rename columns
wide_format.columns = [f'decile_{int(col)}' for col in wide_format.columns]

# Create the long-short portfolio variable
wide_format['decile_10_minus_1'] = wide_format['decile_10'] - wide_format['decile_1']

print(wide_format)

            decile_1  decile_2  decile_3  decile_4  decile_5  decile_6  \
date                                                                     
2018-05-10  0.009156  0.006445  0.005149  0.004445  0.003009  0.001544   
2018-05-11  0.004360  0.003741  0.003064  0.002325  0.002664  0.002180   
2018-05-14  0.000702  0.002271 -0.001826 -0.003615 -0.004997 -0.003564   
2018-05-15  0.002087  0.000403  0.000503  0.001802  0.001820 -0.001974   
2018-05-16  0.011500  0.008576  0.008498  0.008650  0.006110  0.007723   
...              ...       ...       ...       ...       ...       ...   
2020-08-07  0.007975  0.013529  0.014119  0.017976  0.015365  0.016044   
2020-08-10  0.005981  0.011519  0.008043  0.014146  0.010728  0.010953   
2020-08-11 -0.024632 -0.009158  0.004408  0.000815  0.001201  0.003247   
2020-08-12  0.010171  0.002111 -0.001896  0.005662  0.003826  0.006810   
2020-08-13  0.006791  0.001791  0.001583 -0.003183 -0.006434 -0.003980   

            decile_7  decile_8  decil

In [42]:
fama_french = pd.read_csv('./ff_daily.csv')
fama_french['date'] = pd.to_datetime(fama_french['date'], format='%Y-%m-%d')

# Merge the decile data (from the previous step) with the Fama-French factors
data_merged = pd.merge(wide_format, fama_french, on='date', how='inner')

# Regression analysis
if not data_merged.empty and data_merged[['mktrf', 'smb', 'rmw', 'cma', 'hml']].isnull().any().any():
    print("Data contains NaNs in factor columns.")
else:
    X = data_merged[['mktrf', 'smb', 'rmw', 'cma', 'hml']]
    X = sm.add_constant(X)
    y = data_merged['decile_10_minus_1']
    
    # Fit model
    model = sm.OLS(y, X, missing='drop').fit()
    print(model.summary())

    # Calculate the annualized alpha
    annualized_alpha = model.params['const'] * 252
    print(f"Annualized Alpha: {annualized_alpha}")

                            OLS Regression Results                            
Dep. Variable:      decile_10_minus_1   R-squared:                       0.051
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     4.872
Date:                Tue, 07 May 2024   Prob (F-statistic):           0.000237
Time:                        05:01:10   Log-Likelihood:                 1207.5
No. Observations:                 455   AIC:                            -2403.
Df Residuals:                     449   BIC:                            -2378.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0003      0.001     -0.369      0.7

The `decile_10_minus_1` portfolio has an annualized alpha of approximately -0.075, losing to the benchmark adjusted for the Fama-French five factors. The negative alpha suggests that the portfolio isn't providing sufficient returns to justify its risks, particularly in terms of profitability and investment choices. Specifically, its significant negative response to profitability (RMW) and positive response to investment (CMA) imply that it may be overweight in less profitable and higher investment companies, which could be dragging down its overall performance. Also, the insignificant coefficients for size (SMB) and value (HML) indicate that these factors do not effectively explain the portfolio's returns, pointing towards potential issues in stock selection and strategy execution.